In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

import warnings
warnings.filterwarnings("ignore")

# Data Wrangling

## Gathering Data

In [ ]:
# Load & read data
df = pd.read_csv('/kaggle/input/imdb-dataset-of-top-1000-movies-and-tv-shows/imdb_top_1000.csv')
df.head()

## Assessing Data

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
print(df.isnull().sum())

There are 3 columns which have missing values:
- Certificate
- Meta Score
- Gross

In [ ]:
print("Num of duplicated data :", df.duplicated().sum())

In [ ]:
# Check 'Released_Year' values
df['Released_Year'].value_counts()

In [ ]:
df[df['Released_Year']=='PG']

There is a wrong input in 'Released_Year' in the movie 'Apollo 13', which should be the year of release, not a 'PG'; Apollo 13 released year is 1995

In [ ]:
cond = df['Series_Title'] == 'Apollo 13'
df.loc[cond, 'Released_Year']=df.loc[cond, 'Released_Year'].replace('PG', '1995')

In [ ]:
df[df['Series_Title']=='Apollo 13']

In [ ]:
# Check 'Certificate' values 
df['Certificate'].value_counts()

The most certificate of the movie is 'U'

## Data Cleaning

In [ ]:
#Change type data

# Convert runtime to float
df['Runtime'] = df['Runtime'].str.replace(' min', '').astype(int)
# Convert Gross to numeric
df['Gross'] = df['Gross'].str.replace('[\$,]', '', regex=True).astype(float)
# Drop rows with missing values in the 'Certificate' column
df.reset_index(drop=True, inplace=True)
# Check the cleaned DataFrame

In [ ]:
# Check type data
df.dtypes

In [ ]:
# Fill the 'Certificate' null values with the most category appear
df['Certificate'].fillna('U',inplace=True)

In [ ]:
# Fill the 'Meta_score' null values with the average of values
df['Meta_score'].fillna(df['Meta_score'].mean(),inplace=True)

In [ ]:
# Fill the 'Gross' null values with the average value
df['Gross'].fillna(df['Gross'].mean(),inplace=True)
# converting float type to int
df['Gross'] = df['Gross'].astype(int)

In [ ]:
# Check data information
df.info()

In [ ]:
# Create list of Genres
for i in range(df.shape[0]):
    df['Genre'][i] = df['Genre'][i].split(', ')

In [ ]:
# Merge star to list
df['Stars'] = ''
for i in range(df.shape[0]):
    df['Stars'][i] = [df['Star1'][i], df['Star2'][i], df['Star3'][i], df['Star4'][i]]
    
df.drop(['Star1','Star2','Star3','Star4'],axis=1,inplace=True)

In [ ]:
df['Stars'].head()

In [ ]:
df.info()

In [ ]:
df.to_csv('/kaggle/working/df_clean.csv', index=False)

# Exploratory Data Analysis & Visualization

In [ ]:
df.head()

**Q1: Which movies has the highest revenue?**

In [ ]:
df2 = pd.DataFrame(df[['Series_Title', 'Gross']].sort_values(by=['Gross'], ascending=False).reset_index(drop=True))
print('Top 10 Movies with the highest revenue:')
df2.rename(columns={'Series_Title': 'Movie_Title', 'Gross':'Revenue'}, inplace=True)
df2.head(10)

In [ ]:
print('Movie with the highest revenue:\n', df2['Movie_Title'][0])
print('\nwith a revenue by:\n',"${}".format(df2['Revenue'][0]))

In [ ]:
df2_vis=df2.head(10).reset_index(drop=True)
# colors=['#053B50', '#B9B4C7', '#B9B4C7', '#B9B4C7', '#B9B4C7', '#B9B4C7',
#          '#B9B4C7', '#B9B4C7', '#B9B4C7', '#B9B4C7']
plt.figure(figsize=(16, 8))
plt.bar(df2_vis['Movie_Title'], df2_vis['Revenue'])
plt.xticks(rotation=90)
plt.title('Top 10 movies with the highest revenue')
plt.ylabel('Revenue')
plt.show()

**Q2: Which genres are the most common?**

In [ ]:
df3 = pd.DataFrame(df['Genre'].explode().value_counts().reset_index(name='Count'))
df3.head(10)

Drama is the most common genre.

In [ ]:
df3_vis = df3.head(10)
sns.barplot(x='Count', y='Genre', data=df3_vis, orient='h',
            order=df3_vis['Genre'], color='#007FFF')
plt.xlabel('Count')
plt.ylabel('Genre')
plt.title('Top 10 most genres common')
plt.show()

**Q3: How distribution between movie runtime and number of votes?**

In [ ]:
plt.scatter(df['Runtime'],df['No_of_Votes'])
plt.title('Runtime VS No. of Votes')
plt.xlabel('Runtime')
plt.ylabel('No. of Votes')
plt.show()

By the graph, we can conclude that:
- Movies with much higher runtime gets very few votes.
- Only a few movies with average runtime get a high number of votes.


In [ ]:
df.info()

**Q4: Which directors got the highest number of votes on his movies?**

In [ ]:
df4 = pd.DataFrame(df.groupby(by=['Director'])['No_of_Votes'].sum()
                   .reset_index(name='No_of_Votes'))
df4.sort_values(by=['No_of_Votes'], ascending=False).reset_index(drop=True).head(10)

In [ ]:
df4_vis = df4.sort_values(by=['No_of_Votes'], ascending=False).reset_index(drop=True).head(10)
plt.figure(figsize=(10,6))
sns.barplot(x='No_of_Votes', y='Director', data=df4_vis, orient='h',
            order=df4_vis['Director'], color='#007FFF')
plt.xlabel('No. of Votes')
plt.ylabel('Director')
plt.title('Top 10 directors with the highest number of votes in their movies')
plt.show()

**Q5: Number of movies each director created**

In [ ]:
df5 =  pd.DataFrame(df.groupby(by=['Director'])['Series_Title'].count()
                   .reset_index(name='No_of_Movie'))
df5.sort_values(by=['No_of_Movie'], ascending=False).reset_index(drop=True).head(10)

In [ ]:
df5_vis = df5.sort_values(by=['No_of_Movie'], ascending=False).reset_index(drop=True).head(10)
plt.figure(figsize=(10,6))
sns.barplot(x='No_of_Movie', y='Director', data=df5_vis, orient='h',
            order=df5_vis['Director'], color='#007FFF')
plt.xlabel('No. of Movie')
plt.ylabel('Director')
plt.title('Top 10 directors with the highest number of movies')
plt.show()